In [1]:
import sys
sys.path.append('/Users/Jessica/Desktop/Coding/School/2802ICT/Assignment2')
from imports import *


In [16]:

class NN:

    def __init__(self, n_input = 784, n_hidden = 30, n_out = 10, epoch = 30, mini_b = 20, l_rate = 3):
        self.feature_train = []
        self.feature_test = []
        self.target_train = []
        self.target_test = []

        self.N_input = n_input
        self.N_hidden = n_hidden
        self.N_out = n_out
        
        self.epoch = epoch 
        self.minibatch = mini_b
        self.l_rate = l_rate

        self.weights = self.get_weights()


    def arrange_data(self, train_file, test_file):
        
        train_set = self.open_data(train_file)
        test_set = self.open_data(test_file)

        self.target_train, self.feature_train = self.split_data(train_set)
        self.target_test, self.feature_test = self.split_data(test_set)

    def split_data(self, data):
        #seperates a data set into the targets and features
        
        label_set = data["label"]
        target_set = [] 
        
        for row in label_set:
            new_row = self.to_matrix_form(row)
            target_set.append(new_row)   

        df = pd.DataFrame(data)
        feature_set = (df.drop('label', axis=1)/255).astype('float32')

        return np.array(target_set), np.array(feature_set)

    def to_matrix_form(self, val, size = 10):

        arr = np.zeros(size, dtype='float32')
        arr[val] = 1.

        return arr

    def open_data(self, file):
        data_set = pd.read_csv(file)
        return data_set

    def get_weights(self):

        parameters = {
            'W1': np.random.randn(self.N_hidden, self.N_input) * np.sqrt(1. / self.N_hidden),
            'W2': np.random.randn(self.N_out, self.N_hidden) * np.sqrt(1. / self.N_out)
            }
        return parameters

    def forward(self, feature_train):
        
        weights = self.weights
        weights['A0'] = feature_train

        #input layer to hidden layer    
        weights['Z1'] = np.dot(weights['W1'], weights['A0'])
        weights['A1'] = self.sigmoid(weights['Z1'])

        #hidden layer to the output layer
        weights['Z2'] = np.dot(weights['W2'], weights['A1'])
        weights['A2'] = self.softmax(weights['Z2'])


        return weights['A2']

    def backprop(self, feature, output):
        weights = self.weights
        new_weights = {}

        error = 2 * (output - feature) / output.shape[0] * self.softmax(weights['Z2'], True)
        new_weights['W2'] = np.outer(error, weights['A1'])

        error = np.dot(weights['W2'].T, error) * self.sigmoid(weights['Z1'], True)
        new_weights['W1'] = np.outer(error, weights['A0'])

        return new_weights

    def sigmoid(self, z, deriv = False):
        if deriv:
            return (np.exp(-z))/((np.exp(-z)+1)**2)
        return 1.0 / (1.0 + np.exp(-z))

    def softmax(self, z, deriv = False):
       
        exps = np.exp(z - z.max())
        if deriv:
            return exps / np.sum(exps, axis=0) * (1 - exps / np.sum(exps, axis = 0))
        
        return exps / np.sum(exps, axis = 0)

    def train_net(self, feat_train, target_train, feat_test, target_test):

        for i in range(self.epoch):
            for j in range(len(feat_train)):
                output = self.forward(feat_train[j])
                new_weights = self.backprop(target_train[j], output)
                self.update_weights(new_weights)

            accuracy = self.accuracy_calc(feat_test, target_test)
            print('Epoch: {0}, Accuracy: {1}'.format(i+1, accuracy))

    def update_weights(self, new_weights):

        for key, value in new_weights.items():
            self.weights[key] -= self.l_rate * value

                
    def accuracy_calc(self, f_test, t_test):
        predictions = []

        for i in range(len(f_test)):
            out = self.forward(f_test[i])
            pred = np.argmax(out)
            predictions.append(pred == np.argmax(t_test[i]))

        return np.mean(predictions)

    def driver(self, train_file, test_file):
        self.arrange_data(train_file, test_file)
        self.train_net(self.feature_train, self.target_train, self.feature_test, self.target_test)



In [17]:

####################### USER SETUP AREA ####################### 

n_input = 784
n_hidden = 30
n_out = 10
epoch = 30
mini_b = 20
l_rate = 3
    
####################### END OF USER SETUP #######################

nn = NN(n_input, n_hidden, n_out, epoch, mini_b, l_rate)
nn.driver("fashion-mnist_train.csv.gz", "fashion-mnist_test.csv.gz")


Epoch: 1, Accuracy: 0.6329
Epoch: 2, Accuracy: 0.5664
Epoch: 3, Accuracy: 0.4666


<ipython-input-16-85da9b4d3d08>:92: RuntimeWarning: overflow encountered in square
  return (np.exp(-z))/((np.exp(-z)+1)**2)


Epoch: 4, Accuracy: 0.1842
Epoch: 5, Accuracy: 0.1857
Epoch: 6, Accuracy: 0.1872
Epoch: 7, Accuracy: 0.1863


<ipython-input-16-85da9b4d3d08>:93: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))
<ipython-input-16-85da9b4d3d08>:92: RuntimeWarning: overflow encountered in exp
  return (np.exp(-z))/((np.exp(-z)+1)**2)
<ipython-input-16-85da9b4d3d08>:92: RuntimeWarning: invalid value encountered in true_divide
  return (np.exp(-z))/((np.exp(-z)+1)**2)


Epoch: 8, Accuracy: 0.1
Epoch: 9, Accuracy: 0.1
Epoch: 10, Accuracy: 0.1
Epoch: 11, Accuracy: 0.1
Epoch: 12, Accuracy: 0.1
Epoch: 13, Accuracy: 0.1
Epoch: 14, Accuracy: 0.1
Epoch: 15, Accuracy: 0.1
Epoch: 16, Accuracy: 0.1
Epoch: 17, Accuracy: 0.1
Epoch: 18, Accuracy: 0.1
Epoch: 19, Accuracy: 0.1
Epoch: 20, Accuracy: 0.1
Epoch: 21, Accuracy: 0.1
Epoch: 22, Accuracy: 0.1
Epoch: 23, Accuracy: 0.1
Epoch: 24, Accuracy: 0.1
Epoch: 25, Accuracy: 0.1
Epoch: 26, Accuracy: 0.1
Epoch: 27, Accuracy: 0.1
Epoch: 28, Accuracy: 0.1
Epoch: 29, Accuracy: 0.1
Epoch: 30, Accuracy: 0.1
